In [5]:



# %load input_data

import tensorflow as tf
import numpy as np
import os
import pandas as pd
import csv
import numpy as np
from ast import literal_eval


# you need to change this to your data directory
train_dir ='/Users/hmakam200/Desktop/ramya_data/FinalData_256/'
#Function to load the list of image file paths and its labels.
#The following function returns two array lists
#image_list[] is a list of image paths like /Users/hmakam200/Desktop/ramya_data/FinalData_256/2456_right.jpeg
#label_list[] is a list of labels like 0 or 1
def get_files():
    '''    
    Returns:
        list of images and labels
    '''
    #print(tf.__version__)
    print ("Loading files...")
    image_list=[]
    label_list=[]
    trainpath="/Users/hmakam200/Desktop/ramya_data/FinalData_256/"
    count=0
    drimage=0
    with open('images10.csv') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:                        
            image_list.append(trainpath+row[0]+".jpeg")
            temp=literal_eval(row[1])            
            label_list.append(temp)
            count=count+1
            if temp==1:
                drimage=drimage+1  
            #if count==6000:
                #break
                
    
    print ("DR affected images number ",+drimage)
    print ("Files list loaded.")
    return image_list, label_list

#Function to generate batches of images and labels
def get_batch(image, label, image_W, image_H, batch_size, capacity):
    '''
    Args:
        image: list type
        label: list type
        image_W: image width
        image_H: image height
        batch_size: batch size
        capacity: the maximum elements in queue
    Returns:
        image_batch: 4D tensor [batch_size, width, height, 3], dtype=tf.float32
        label_batch: 1D tensor [batch_size], dtype=tf.int32
    '''       
    image = tf.cast(image, tf.string)
    label = tf.to_int32(label)
    
    input_queue = tf.train.slice_input_producer([image, label]) 
    label = input_queue[1]
    
    image_contents = tf.read_file(input_queue[0])
    image = tf.image.decode_jpeg(image_contents, channels=3)

    # data argumentation should go to here #####
    
    image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H) 
    
    
    image=tf.image.random_flip_left_right(image)
    image=tf.image.random_brightness(image, max_delta=0.3)
    image= tf.image.random_contrast(image, 0.8, 1.2)    
    image=tf.image.random_flip_up_down(image)
    image=tf.image.random_hue(image,max_delta=0.3)
    image = tf.image.per_image_standardization(image)
    
    image_batch, label_batch = tf.train.batch([image, label],
                                                batch_size= batch_size,
                                                num_threads= 2, 
                                                capacity = capacity)
    
    
    label_batch = tf.reshape(label_batch, [batch_size])
    image_batch = tf.cast(image_batch, tf.float32)
    
    return image_batch, label_batch
get_files()

Loading files...
DR affected images number  1047
Files list loaded.


(['/Users/hmakam200/Desktop/ramya_data/FinalData_256/19_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/19_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/20_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/20_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/21_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/21_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/46_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/46_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/47_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/47_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/49_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/49_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/65_left.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/FinalData_256/65_right.jpeg',
  '/Users/hmakam200/Desktop/ramya_data/Fi

In [6]:
# %load model
import tensorflow as tf

#Function to build the convolution model
def inference(images, batch_size, n_classes,keep_prob):
    '''Build the model
    Args:
        images: image batch, 4D tensor, tf.float32, [batch_size, width, height, channels]
    Returns:
        output tensor with the computed logits, float, [batch_size, n_classes]
    '''
    #conv1, shape = [kernel size, kernel size, channels, kernel numbers]
    
    with tf.variable_scope('conv1') as scope:
        weights = tf.get_variable('weights', 
                                  shape = [3,3,3, 16],
                                  dtype = tf.float32, 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1,dtype=tf.float32))
        biases = tf.get_variable('biases', 
                                 shape=[16],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name= scope.name)
    
    #pool1 and norm1   
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1],strides=[1,2,2,1],
                               padding='SAME', name='pooling1')
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0,
                          beta=0.75,name='norm1')
    
    #conv2
    with tf.variable_scope('conv2') as scope:
        weights = tf.get_variable('weights',
                                  shape=[3,3,16,16],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.1,dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[16], 
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(norm1, weights, strides=[1,1,1,1],padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name='conv2')
    
    
    #pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0,
                          beta=0.75,name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1],
                               padding='SAME',name='pooling2')
    
    
    #local3
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = tf.get_variable('weights',
                                  shape=[dim,128],
                                  initializer=tf.truncated_normal_initializer(stddev=0.005,dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[128],
                                 dtype=tf.float32, 
                                 initializer=tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)    
    
    #local4
    with tf.variable_scope('local4') as scope:
        weights = tf.get_variable('weights',
                                  shape=[128,128],
                                  dtype=tf.float32, 
                                  initializer=tf.truncated_normal_initializer(stddev=0.005,dtype=tf.float32))
        biases = tf.get_variable('biases',
                                 shape=[128],
                                 dtype=tf.float32,
                                 initializer=tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')
     
        
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.get_variable('softmax_linear',
                                  shape=[128, n_classes],
                                  dtype=tf.float32,
                                  initializer=tf.truncated_normal_initializer(stddev=0.005,dtype=tf.float32))
        biases = tf.get_variable('biases', 
                                 shape=[n_classes],
                                 dtype=tf.float32, 
                                 initializer=tf.constant_initializer(0.1))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
    
    return softmax_linear

def losses(logits, labels):
    '''Compute loss from logits and labels
    Args:
        logits: logits tensor, float, [batch_size, n_classes]
        labels: label tensor, tf.int32, [batch_size]
        
    Returns:
        loss tensor of float type
    '''
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.summary.scalar(scope.name+'/loss', loss)
    return loss


def trainning(loss, learning_rate):
    '''Training ops, the Op returned by this function is what must be passed to 
        'sess.run()' call to cause the model to train.
        
    Args:
        loss: loss tensor, from losses()
        
    Returns:
        train_op: The op for trainning
    '''
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op


def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
      Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size], with values in the
        range [0, NUM_CLASSES).
      Returns:
        A scalar int32 tensor with the number of examples (out of batch_size)
        that were predicted correctly.
    """
    
    
    with tf.variable_scope('accuracy') as scope:
        
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)         
        tf.summary.scalar(scope.name+'/accuracy', accuracy)
        
    return accuracy



In [8]:
import os
import numpy as np
import tensorflow as tf
import time
from datetime import datetime

#%%

N_CLASSES = 2
IMG_W = 256  
IMG_H = 256
BATCH_SIZE = 16

CAPACITY = 2000
MAX_STEP = 10000 # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001 # with current parameters, it is suggested to use learning rate<0.0001


#%%
def run_training():
    
    # you need to change the directories to yours.
    
    logs_train_dir = '/Users/hmakam200/personal/tensorflow/ramya-capstone/savedModel/'
    
    train, train_label = get_files()
    
    train_batch, train_label_batch = get_batch(train,train_label,IMG_W,IMG_H,BATCH_SIZE,CAPACITY)      
    train_logits = inference(train_batch, BATCH_SIZE, N_CLASSES,keep_prob=0.7)
    train_loss = losses(train_logits, train_label_batch)        
    train_op = trainning(train_loss, learning_rate)
    train__acc =evaluation(train_logits, train_label_batch)
       
    summary_op = tf.summary.merge_all()
    sess = tf.Session()
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    f=open("/Users/hmakam200/personal/tensorflow/ramya-capstone/logs.txt",'w')
    tic=time.clock()
    total=time.clock()
    try:
        for step in np.arange(MAX_STEP):
            if coord.should_stop():
                    break
            _, tra_loss, tra_acc = sess.run([train_op, train_loss, train__acc])           
               
            if step % 50 == 0:
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)
                f.write("step :%s  loss : %f  accuracy:  %f\n" % (step, tra_loss,tra_acc*100.0))               

            if step==9999 :
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)
                toc=time.clock()
                print("Process Time for total iterations: {:.3f}".format(toc - total))               
            
            if step % 1000 == 0 or (step + 1) == MAX_STEP:
                checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
                toc=time.clock()
                print("Process Time for 1000 iterations: {:.3f}".format(toc - tic))
                tic=toc
                print (str(datetime.now()))
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()
        
    coord.join(threads)
    sess.close()
    f.close()



In [9]:
tf.reset_default_graph() 
print ("Training started.")

run_training()
print ("training ended.")

Training started.
Loading files...
DR affected images number  1047
Files list loaded.
Step 0, train loss = 0.70, train accuracy = 6.25%
Process Time for 1000 iterations: 20.673
2017-11-17 11:52:07.121106
Step 50, train loss = 0.49, train accuracy = 81.25%
Step 100, train loss = 0.41, train accuracy = 87.50%
Step 150, train loss = 0.57, train accuracy = 75.00%
Step 200, train loss = 0.36, train accuracy = 87.50%
Step 250, train loss = 0.37, train accuracy = 87.50%
Step 300, train loss = 0.36, train accuracy = 93.75%
Step 350, train loss = 0.25, train accuracy = 93.75%
Step 400, train loss = 0.50, train accuracy = 81.25%
Step 450, train loss = 0.39, train accuracy = 87.50%
Step 500, train loss = 0.38, train accuracy = 87.50%
Step 550, train loss = 0.18, train accuracy = 100.00%
Step 600, train loss = 0.39, train accuracy = 87.50%
Step 650, train loss = 0.87, train accuracy = 56.25%
Step 700, train loss = 0.27, train accuracy = 93.75%
Step 750, train loss = 0.40, train accuracy = 87.50%
S

Step 7050, train loss = 0.75, train accuracy = 68.75%
Step 7100, train loss = 0.34, train accuracy = 87.50%
Step 7150, train loss = 0.58, train accuracy = 68.75%
Step 7200, train loss = 0.34, train accuracy = 87.50%
Step 7250, train loss = 0.42, train accuracy = 81.25%
Step 7300, train loss = 0.41, train accuracy = 81.25%
Step 7350, train loss = 0.34, train accuracy = 87.50%
Step 7400, train loss = 0.30, train accuracy = 87.50%
Step 7450, train loss = 0.42, train accuracy = 75.00%
Step 7500, train loss = 0.34, train accuracy = 93.75%
Step 7550, train loss = 0.46, train accuracy = 87.50%
Step 7600, train loss = 0.47, train accuracy = 81.25%
Step 7650, train loss = 0.56, train accuracy = 75.00%
Step 7700, train loss = 0.34, train accuracy = 87.50%
Step 7750, train loss = 0.52, train accuracy = 75.00%
Step 7800, train loss = 0.46, train accuracy = 81.25%
Step 7850, train loss = 0.59, train accuracy = 68.75%
Step 7900, train loss = 0.15, train accuracy = 100.00%
Step 7950, train loss = 0.5

In [11]:
# %load input_data

import tensorflow as tf
import numpy as np
import os
import pandas as pd
import csv
import numpy as np
from ast import literal_eval


# you need to change this to your data directory
#train_dir ='F:/Udacity ML/git/udacity-capstone'
#function to read test image paths and labels
def get_files_test():
    '''   
    Returns:
        list of images and labels
    '''
    #print(tf.__version__)
    
    image_list=[]
    label_list=[]
    testpath="/Users/hmakam200/Desktop/ramya_data/FinalData_256/"
    with open('testdata.csv') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
                        
            image_list.append(testpath+row[0]+".jpeg")
            temp=literal_eval(row[1])            
            label_list.append(temp)    
    
    return image_list, label_list

def get_batch_test(image, label, image_W, image_H, batch_size, capacity):
    '''
    Args:
        image: list type
        label: list type
        image_W: image width
        image_H: image height
        batch_size: batch size
        capacity: the maximum elements in queue
    Returns:
        image_batch: 4D tensor [batch_size, width, height, 3], dtype=tf.float32
        label_batch: 1D tensor [batch_size], dtype=tf.int32
    '''
    
    image = tf.cast(image, tf.string)#cast image list values to string tensors
    label = tf.to_int32(label)  


    # make an input queue
    input_queue = tf.train.slice_input_producer([image, label])    
    label = input_queue[1]
    
    image_contents = tf.read_file(input_queue[0])
    
    image = tf.image.decode_jpeg(image_contents, channels=3)    
    
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)    
    
    
    
          
    image_batch, label_batch = tf.train.batch([image, label],
                                                batch_size= batch_size,
                                                num_threads= 2, 
                                                capacity = capacity)
    
    
    
    label_batch = tf.reshape(label_batch, [batch_size])
    image_batch = tf.cast(image_batch, tf.float32)
    
    return image_batch, label_batch


In [41]:
#%% Evaluate one image
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

def get_one_image(test_images_path,ind):
    '''Randomly pick one image from training data
    Return: ndarray
    '''
    #n = len(test_images_path)
    #ind = np.random.randint(0, n)
    img_dir = test_images_path[ind]

    image = Image.open(img_dir)
    plt.imshow(image)
    image = image.resize([256, 256])
    image = np.array(image)
    return image

def evaluate_test():
    
    IMG_W = 256 
    IMG_H = 256
    BATCH_SIZE = 2554
    CAPACITY = 100
    N_CLASSES=2
    tf.reset_default_graph() 
    ind=0
    correct=0
    wrong=0
    tp=0
    fp=0
    tn=0
    fn=0
    MAX_STEP=1
   
    test, test_label = get_files_test()
    
    
    test_batch, test_label_batch = get_batch_test(test,test_label,IMG_W,IMG_H,BATCH_SIZE,CAPACITY)      
    test_logits = inference(test_batch, BATCH_SIZE, N_CLASSES,keep_prob=1)
    test__acc =evaluation(test_logits, test_label_batch)
       
    sess = tf.Session()
    #train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)   
    
          
    
    init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
 
                       
    #saver = tf.train.Saver()
     
    logs_train_dir = '/Users/hmakam200/personal/tensorflow/ramya-capstone/savedModel/' 
    try:
        for step in np.arange(MAX_STEP):
            if coord.should_stop():
                    break
            print("Reading checkpoints...")
            ckpt = tf.train.get_checkpoint_state(logs_train_dir)
            if ckpt and ckpt.model_checkpoint_path:            
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                saver.restore(sess, ckpt.model_checkpoint_path)
                print('Loading success, global_step is %s' % global_step)
            else:
                print('No checkpoint file found')
            
            test_prediction, test_acc = sess.run([test_logits, test__acc])
            
            #print ()
            print (test_acc)
            print(np.shape(test_prediction))
               
    except tf.errors.OutOfRangeError:
        print('Done testing -- epoch limit reached')
    finally:
        coord.request_stop()
        
    coord.join(threads)
    sess.close()

    
evaluate_test()
print ("Done")

Reading checkpoints...
INFO:tensorflow:Restoring parameters from /Users/hmakam200/personal/tensorflow/ramya-capstone/savedModel/model.ckpt-9999
Loading success, global_step is 9999
0.80176
(2554, 2)
Done
